# Lunaモデルの学習

### Lunaサイト
https://luna16.grand-challenge.org/Description/

### データセット概要
アーカイブを展開した状態のすべてのデータで約220GBを必要とする.  
+ annotations.csv
+ candidates.csv
+ sampleSubmission.csv
+ candidates_V2.zip
+ evaluationScript.zip
+ seg-lungs-LUNA16.zip
1. subset0.zip
2. subset1.zip
3. subset2.zip
4. subset3.zip
5. subset4.zip
6. subset5.zip
7. subset6.zip
8. subset7.zip
9. subset8.zip
10. subset10.zip

### subsetの中身
+ series_uidをユニークIDとして*.mhdと*.rawの2種類のファイルがある

In [4]:

import datetime
from p2_ct_project.util.logconf import logging
log = logging.getLogger('nb')
# log.setLevel(logging.WARN)
log.setLevel(logging.INFO)
log.setLevel(logging.DEBUG)

from p2_ct_project.util.util import importstr

# run_everything
def run(app, *argv):
    argv = list(argv)
    argv.insert(0, '--num-workers=8')
    # 4コア8スレッドのCPUを仮定
    log.info("Running: {}({!r}).main()".format(app, argv))
    print("*app.rsplit('.', 1) : ", *app.rsplit('.', 1))

    app_cls = importstr(*app.rsplit('.', 1)) # __import__を実行
    app_cls(argv).main()

    log.info("Finished: {}.({!r}).main()".format(app, argv))

In [5]:
import os
import shutil

# clean up any old data that might be around.
# We don't call this by default because it's destructive, 
# and would waste a lot of time if it ran when nothing 
# on the application side had changed.
tag_version = "unversioned"
cache_dir = f"F:/Luna16/cache/{tag_version}"
def cleanCache():
    shutil.rmtree(cache_dir)
    os.mkdir(cache_dir)

# cleanCache()

Fancacheのclose

In [6]:
from p2_ct_project.p2ch10.dataset_luna import raw_cache
raw_cache.close()

from p2_ct_project.p2ch12.mod_dataset_luna import raw_cache
raw_cache.close()

from p2_ct_project.p2ch13.dataset_luna_seg import raw_cache
raw_cache.close()

### 各チャプターで共通の学習設定

In [7]:
experiment_epochs = 2
training_epochs = 20
final_epochs = 50
seg_epochs = 10

# Chapter11

### デフォルト設定で学習

In [8]:
# run('p2_ct_project.p2ch11.prepcache_luna.LunaPrepCacheApp', '--datasetdir=E:/Luna16')

In [9]:
# run('p2_ct_project.p2ch11.training_luna.LunaTrainingApp', '--datasetdir=E:/Luna16', '--epochs=1')

# Chapter12
+ 陽性サンプルと陰性サンプルの均一化

## 結節データをかさ増しする
+ LunaDatasetの結節候補データは, 結節でないデータと結節データの比率が均等でない

### 課題
+ ラベルが均等でないデータセットでは, 学習時にデータ比率が大きいラベル側にNNモデルが引っ張られ, 推論でデータ比率の大きなラベルばかりを推論結果として出力してしまう.

### 解決策
1. 学習時は, 各ラベルのデータ数を均等にする. 
2. 損失関数で, 比率の小さいラベルデータに対して更新勾配(誤差)を大きくする処理を追加する

#### 今回は比率の小さいラベルデータをかさ増しする

### 結節データ(比率の小さいラベルデータ)のかさ増しをする方法
1. PytorchのDataLoader関数の`sampler=...`オプションを使用する方法. <br> データセットの元々のイテレーション順序を上書きすることができ, <br> 特定のサンプルの抽出を制限したり, 逆に複数回抽出することが可能. これはデータセット自体をコントロールできない場合に非常に有効.
2. 直接データセットをコントロールできる場合, Datasetクラスのサブクラスの中で, 必要となるデータを再構成して, かさ増しする.

### データセット内でクラスバランスを調整する
+ LunaDataset内に追加の実装を行い, 訓練セットの陽性サンプル数と陰性サンプル数が1対1になるように変更する

In [10]:
# run('p2_ct_project.p2ch12.prepcache_luna.LunaPrepCacheApp', '--datasetdir=E:/Luna16')

In [11]:
# run('p2_ct_project.p2ch12.mod_training_luna.LunatrainingApp', '--datasetdir=E:/Luna16', '--epochs=1', 'unbalanced')

In [12]:
# データサンプルの調整
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', '--datasetdir=E:/Luna16', f'--epochs=1', '--balanced', 'balanced')

In [13]:
# import numpy as np
# a = np.empty((1, 100), dtype=np.float32) > 0.5
# print(a)
# ans = np.sum(a)
# print(ans)

### データ拡張の結果を比較する

In [14]:
# データサンプルの調整
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', '--datasetdir=E:/Luna16', f'--epochs={training_epochs}', '--balanced', 'balanced')

In [15]:
# データ拡張　フリップ
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augment-flip', 'flip')

In [16]:
# データ拡張 平行移動
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augment-offset', 'offset')

In [17]:
# データ拡張 スケール
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augment-scale', 'scale')

In [18]:
# データ拡張　回転
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augment-rotate', 'rotate')

In [19]:
# データ拡張　ノイズ付加
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augment-noise', 'noise')

In [20]:
# データ拡張　フリップ, 平行移動, スケール, 回転, ノイズ付加
# run('p2_ct_project.p2ch12.mod_training_luna.LunaTrainingApp', f'--epochs={training_epochs}', '--balanced', '--augmented', 'fully-augmented')

# Chapter13

セマンティックセグメンテーションモデルによる結節候補の推論

In [21]:
# 加工データセットのキャッシュを作成
# run('p2_ct_project.p2ch13.prepcache_luna_seg.LunaSegPrepCacheApp')

In [23]:
datasetdir = 'E:/Luna16'
run('p2_ct_project.p2ch13.training_luna_seg.SegmentationTrainingApp', 
    f'--epochs={experiment_epochs}', 
    '--augmented', 
    f'--datasetdir={datasetdir}'
    )

2023-06-05 00:38:45,901 INFO     pid:8296 nb:015:run Running: p2_ct_project.p2ch13.training_luna_seg.SegmentationTrainingApp(['--num-workers=8', '--epochs=2', '--augmented', '--datasetdir=E:/Luna16']).main()
2023-06-05 00:38:45,922 INFO     pid:8296 p2_ct_project.p2ch13.training_luna_seg:169:initModel Using CUDA; 1 devices.
2023-06-05 00:38:46,004 INFO     pid:8296 p2_ct_project.p2ch13.training_luna_seg:238:main Starting SegmentationTrainingApp, Namespace(batch_size=16, num_workers=8, epochs=2, augmented=True, augment_flip=False, augment_offset=False, augment_scale=False, augment_rotate=False, augment_noise=False, tb_prefix='p2ch13', comment='none', datasetdir='E:/Luna16')


*app.rsplit('.', 1) :  p2_ct_project.p2ch13.training_luna_seg SegmentationTrainingApp
module:  <module 'p2_ct_project' (namespace)>


2023-06-05 00:38:51,947 INFO     pid:8296 p2ch13.dataset_luna_seg:393:__init__ <p2ch13.dataset_luna_seg.TrainingLuna2dSegmentationDataset object at 0x000002A3869558E0>: 799 training seires, 8343 slices, 1028 nodules
2023-06-05 00:38:52,089 INFO     pid:8296 p2ch13.dataset_luna_seg:393:__init__ <p2ch13.dataset_luna_seg.Luna2dSegmentationDataset object at 0x000002A38709E070>: 89 validation seires, 1122 slices, 154 nodules
2023-06-05 00:38:52,089 INFO     pid:8296 p2_ct_project.p2ch13.training_luna_seg:246:main Epoch 1 of 2, 18750/71 batches of size 16*1
2023-06-05 00:38:53,126 WARNING  pid:8296 util.util:226:enumerateWithEstimate E1 Training  ----/18750, starting
2023-06-05 00:39:19,683 INFO     pid:8296 util.util:243:enumerateWithEstimate E1 Training   256/18750, done at 2023-06-05 00:41:32, 0:02:14
2023-06-05 00:39:32,502 INFO     pid:8296 util.util:243:enumerateWithEstimate E1 Training  2048/18750, done at 2023-06-05 00:41:32, 0:02:14
2023-06-05 00:41:15,365 INFO     pid:8296 util.uti

In [24]:
datasetdir = 'E:/Luna16'
seg_epochs = 10
run('p2_ct_project.p2ch13.training_luna_seg.SegmentationTrainingApp', 
    f'--epochs={seg_epochs}', 
    '--augmented', 
    'final-seg', 
    f'--datasetdir={datasetdir}'
    )

2023-06-05 00:46:03,718 INFO     pid:8296 nb:015:run Running: p2_ct_project.p2ch13.training_luna_seg.SegmentationTrainingApp(['--num-workers=8', '--epochs=10', '--augmented', 'final-seg', '--datasetdir=E:/Luna16']).main()
2023-06-05 00:46:03,721 INFO     pid:8296 p2_ct_project.p2ch13.training_luna_seg:169:initModel Using CUDA; 1 devices.
2023-06-05 00:46:03,725 INFO     pid:8296 p2_ct_project.p2ch13.training_luna_seg:238:main Starting SegmentationTrainingApp, Namespace(batch_size=16, num_workers=8, epochs=10, augmented=True, augment_flip=False, augment_offset=False, augment_scale=False, augment_rotate=False, augment_noise=False, tb_prefix='p2ch13', comment='final-seg', datasetdir='E:/Luna16')


*app.rsplit('.', 1) :  p2_ct_project.p2ch13.training_luna_seg SegmentationTrainingApp
module:  <module 'p2_ct_project' (namespace)>


2023-06-05 00:46:04,014 INFO     pid:8296 p2ch13.dataset_luna_seg:393:__init__ <p2ch13.dataset_luna_seg.TrainingLuna2dSegmentationDataset object at 0x000002A38709EB80>: 799 training seires, 8343 slices, 1028 nodules
2023-06-05 00:46:04,188 INFO     pid:8296 p2ch13.dataset_luna_seg:393:__init__ <p2ch13.dataset_luna_seg.Luna2dSegmentationDataset object at 0x000002A3869554F0>: 89 validation seires, 1122 slices, 154 nodules
2023-06-05 00:46:04,188 INFO     pid:8296 p2_ct_project.p2ch13.training_luna_seg:246:main Epoch 1 of 10, 18750/71 batches of size 16*1
2023-06-05 00:46:04,402 WARNING  pid:8296 util.util:226:enumerateWithEstimate E1 Training  ----/18750, starting
2023-06-05 00:46:31,204 INFO     pid:8296 util.util:243:enumerateWithEstimate E1 Training   256/18750, done at 2023-06-05 00:48:51, 0:02:22
2023-06-05 00:46:44,593 INFO     pid:8296 util.util:243:enumerateWithEstimate E1 Training  2048/18750, done at 2023-06-05 00:48:49, 0:02:20
2023-06-05 00:48:31,776 INFO     pid:8296 util.ut